In [6]:
import numpy as np
np.random.seed(5)
#gaussian distribution sampler: normal(mean=0.0, variance_sqrt=1.0, size=None) s = np.random.normal(0,1)
#teacher model
#randomly generate z=(x,y) with x evenly sampled from (0,10) and y = 1.5 x + 5, a random noise is added. A total of N samples are drawn.
#dimension of x: d
d = 10
#number of training samples
N = 5
#dimension of hypothesis space
p = 10

#point-wise activate function f:tanh
#variance of random noise added to y
sigma = 0.0
#samples X(N*d)
X = np.random.normal(0,1,(N,d))
#random feature matrix F(d*p)
F = np.random.normal(0,1,(d,p))
#teacher parameter w(p) with lambda = ? until each dim of Y~1e0
lambda_ = 0.0001
#w_0 = np.random.normal(0,sigma/np.sqrt(lambda_*N),p)
w_0 = np.random.normal(0,1,p)
#X after the random feature matrix
X_rf = np.tanh(X.dot(F)/np.sqrt(d))
Y_pure = X_rf.dot(w_0)
Y = Y_pure + np.random.normal(0,sigma,N)
print(Y_pure)
print(Y)

[-0.55095189 -0.88060408 -1.3119232   0.78306986  0.35476296]
[-0.55095189 -0.88060408 -1.3119232   0.78306986  0.35476296]


In [7]:
#empirical risk
#L_S(w)
def L_S(w):
    diff = Y-X_rf.dot(w)
    Nloss = diff.dot(diff)
    return Nloss/N

def grad_L_S(w):
    return -2/N*X_rf.T.dot(Y-X_rf.dot(w))

grad_L_S(w_0+0.1)#when N is small, diffusion.

array([-0.22610429, -0.07479486, -0.12451011,  0.05415974,  0.23965182,
        0.13283303,  0.1449693 ,  0.17232128,  0.226123  ,  0.17689214])

In [8]:
#minus log distribution: f
#prar beta(also change h if change this)
beta = 1000
#para sigma_q, can be set according to N.
sigma_q = 1
def f(w): 
    return beta*L_S(w)+(1/2/sigma_q**2)*(w.dot(w))

def grad_f(w):
    grad = beta*grad_L_S(w)
    return grad

In [9]:
#MCMC
#stepsize h(h*beta=0.01)
h = 0.0001
w = np.zeros(p)
for i in range(100000):
    grad_f_w = grad_f(w)
    proposal_state = w-h*grad_f_w
    w = proposal_state
#print(w)
#print(w_0)
diviant = np.linalg.norm((w-w_0)/np.average(w_0))

(w-w_0)/np.average(w_0)

print(w[0:10])
print(w_0[0:10])
print(np.linalg.norm(w))
print(np.linalg.norm(w_0))
diviant

[ 0.29979583 -0.4251767  -0.24561404 -1.0388348  -0.27759258  0.23661841
  1.24651599  0.31233101 -0.88298462  0.23330221]
[ 1.40216662  0.46510099 -1.06503262  0.39042061  0.30560017  0.52184949
  2.23327081 -0.0347021  -1.27962318  0.03654264]
2.007149026554329
3.2345486345966656


8.52423436498875

In [10]:
#a = np.linalg.inv(X_rf).dot(Y)
#print(np.linalg.norm(a))
#a#